# Eurybia - dataprep for US car accidents
This notebook describes the data preparation leading to the dataset in "US_Accidents_extract.csv", used in some of our tutorials.  


The original dataset was taken from the Kaggle [US car accidents dataset](https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents).

---
Acknowledgements
- Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, and Rajiv Ramnath. “A Countrywide Traffic Accident Dataset.”, 2019.
- Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, Radu Teodorescu, and Rajiv Ramnath. "Accident Risk Prediction based on Heterogeneous Sparse Data: New Dataset and Insights." In proceedings of the 27th ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems, ACM, 2019.
---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import category_encoders as ce

### Extract the zipped dataset if you haven't already done so

In [2]:
# from zipfile import ZipFile
# with ZipFile('/tmp/archive.zip', 'r') as zipObj:
#     zipObj.extractall()

### Load it up

In [2]:
data = pd.read_csv('US_Accidents_March23.csv')

In [3]:
print(data.shape)
print(data.columns)

(7728394, 46)
Index(['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
       'Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')


In [4]:
feats_to_keep = ['Start_Lat','Start_Lng','Distance(mi)','Temperature(F)','Humidity(%)','Visibility(mi)',
 'day_of_week_acc','Nautical_Twilight','season_acc','target','target_multi','year_acc','Description']

### Create targets column  
Here we regroup the severity modalities into two classes to create a binary target column : benign to moderate severity (<= 2) on one side, serious and above on the other (>2)

In [5]:
print(data.Severity.value_counts())
cond = [data.Severity <= 2]
choice = ['0']
data['target'] = np.select(cond, choice, default = '1')
data['target'].value_counts(normalize = True)*100

Severity
2    6156981
3    1299337
4     204710
1      67366
Name: count, dtype: int64


target
0    80.538686
1    19.461314
Name: proportion, dtype: float64

In [6]:
data = data.rename(columns={'Severity':'target_multi'})

### Rework the dates  
Here we build a "day of week", a "season" and a "year" feature. This will help us detect and analyze bias or trends that occur on those timescales.  
For example, we can then measure the drift between two same seasons of consecutive years to avoid seasonal bias.  
We could also aggregate by year and mesure the drift from year to year.

In [21]:
date = ['Start_Time']
for d in date:    
    data[d] = pd.to_datetime(data[d])

In [22]:
data['month_acc'] = data['Start_Time'].dt.month
data['day_of_week_acc'] = data['Start_Time'].dt.dayofweek
data['year_acc'] = data['Start_Time'].dt.year
cond = [data.month_acc.isin([12,1,2]), 
       data.month_acc.isin([3,4,5]), 
       data.month_acc.isin([6,7,8]), 
       data.month_acc.isin([9,10,11])]
choix = ['winter','spring','summer','autumn']
data['season_acc'] = np.select(cond, choix, default = 'NR')

### Managing missing values  
The following short analysis, using the "year" feature we just created, lets us see how the ratio of missing values evolved over time.  

In [23]:
missing_val = pd.DataFrame()
year = np.unique(data.year_acc)
for y in year:
    sub = data[data.year_acc == y]
    missing_val_y = pd.DataFrame(sub.isnull().sum().sort_values(ascending=False)/sub.shape[0]*100)
    missing_val_y.columns = ['taux_miss_'+str(y)]
    missing_val = pd.concat([missing_val, missing_val_y], axis = 1)

In [24]:
missing_val['filtre'] = missing_val.taux_miss_2016+missing_val.taux_miss_2017+missing_val.taux_miss_2018+missing_val.taux_miss_2019+missing_val.taux_miss_2020+missing_val.taux_miss_2021+missing_val.taux_miss_2022
missing_val[missing_val.filtre > 0][['taux_miss_2016','taux_miss_2017','taux_miss_2018','taux_miss_2019','taux_miss_2020','taux_miss_2021','taux_miss_2022']]

,taux_miss_2016,taux_miss_2017,taux_miss_2018,taux_miss_2019,taux_miss_2020,taux_miss_2021,taux_miss_2022
Precipitation(in),90.928409,88.028013,86.740256,22.976036,5.293181,4.195420,3.658256
Wind_Chill(F),89.963025,84.850848,81.230566,15.642097,3.406782,2.696526,3.092737
End_Lat,68.463151,76.228427,81.222731,72.506741,39.887931,28.782359,13.400762
End_Lng,68.463151,76.228427,81.222731,72.506741,39.887931,28.782359,13.400762
Wind_Speed(mph),18.846651,18.770967,18.891100,6.294646,3.033388,2.445783,2.744869
Visibility(mi),2.137427,2.413058,2.359233,1.930624,2.574914,2.066183,2.380377
Weather_Condition,2.082902,2.443973,2.408929,1.909876,2.542512,2.066375,2.195124
Humidity(%),1.706096,2.088309,2.112542,1.907885,2.819038,2.172338,2.332376
Temperature(F),1.569783,2.003919,2.015948,1.794503,2.626487,2.047766,2.196826
Pressure(in),1.238009,1.768434,1.850741,1.439061,2.198211,1.784873,1.879994


$\require{color}$
$\colorbox{red}{The percentage of missing values, aggregated by year, is far from constant. This is a preliminary sign of data drift.}$

In [25]:
data = data.dropna(subset=["Nautical_Twilight"])

### Final dataset features

In [26]:
data = data[feats_to_keep]

### Quantitative features  

In [27]:
for v in ['Distance(mi)','Temperature(F)','Humidity(%)','Visibility(mi)']:
    data[v] = np.round(data[v],0)

In [28]:
data['Start_Lat'] = np.round(data['Start_Lat'],1)
data['Start_Lng'] = np.round(data['Start_Lng'],1)

### Sampling  
For the purpose of our tutorials, a sample size of ~50000 is sufficient.  
The following few steps reduce the sample size down to about this number, and balance the number of samples per year, in an effort to reduce this source of bias before training a model or producing a quantitative analysis.

In [29]:
sampled_data = pd.DataFrame()
annee = np.unique(data.year_acc)
for a in annee:
    sub = data[data.year_acc == a]
    sub = sub.reset_index(drop = True)
    tir = np.random.choice(a = sub.shape[0], size = 50000//len(annee)+1, replace = False)
    sampled_data = pd.concat([sampled_data, sub.iloc[tir,:]], axis = 0)
    sampled_data = sampled_data.reset_index(drop = True)

In [30]:
sampled_data = sampled_data.iloc[np.random.choice(size = 50000, a = sampled_data.index, replace = False),:]
sampled_data = sampled_data.reset_index(drop  = True)

### Let us have a final look at our data :

In [31]:
sampled_data.head(3)

,Start_Lat,Start_Lng,Distance(mi),Temperature(F),Humidity(%),Visibility(mi),day_of_week_acc,Nautical_Twilight,season_acc,target,target_multi,year_acc,Description
0,26.5,-81.8,0.0,77.0,50.0,10.0,1,Day,spring,0,2,2017,Accident on Winged Foot Dr at Lee Rd.
1,29.5,-98.5,0.0,50.0,83.0,10.0,4,Day,winter,0,2,2017,Ramp to I-410 - Accident.
2,33.9,-118.3,0.0,49.0,36.0,10.0,2,Day,winter,1,3,2018,Right hand shoulder blocked due to accident on...


In [32]:
sampled_data.describe()

,Start_Lat,Start_Lng,Distance(mi),Temperature(F),Humidity(%),Visibility(mi),day_of_week_acc,target_multi,year_acc
count,50000.000000,50000.000000,50000.000000,48967.000000,48900.000000,48843.000000,50000.000000,50000.000000,50000.000000
mean,36.331194,-95.275684,0.484060,60.584557,65.307587,9.056938,2.558260,2.239600,2019.500040
std,5.025208,17.514641,1.684186,18.976629,22.608926,2.736458,1.785259,0.501275,2.291354
min,24.700000,-124.500000,0.000000,-89.000000,3.000000,0.000000,0.000000,1.000000,2016.000000
25%,33.500000,-117.400000,0.000000,48.000000,49.000000,10.000000,1.000000,2.000000,2017.000000
50%,35.900000,-88.200000,0.000000,62.000000,67.000000,10.000000,2.000000,2.000000,2020.000000
75%,40.200000,-80.600000,0.000000,75.000000,85.000000,10.000000,4.000000,2.000000,2022.000000
max,49.000000,-68.300000,102.000000,117.000000,100.000000,80.000000,6.000000,4.000000,2023.000000


### Write the sample to disk

In [33]:
sampled_data.to_csv('US_Accidents_extract.csv', index = False)

In [34]:
sampled_data.loc[sampled_data["year_acc"] == 2016]

,Start_Lat,Start_Lng,Distance(mi),Temperature(F),Humidity(%),Visibility(mi),day_of_week_acc,Nautical_Twilight,season_acc,target,target_multi,year_acc,Description
4,42.4,-83.2,0.0,40.0,82.0,10.0,3,Day,autumn,0,2,2016,Accident on 7 Mile Rd at Lindsay St.
7,41.0,-73.8,1.0,37.0,46.0,10.0,0,Day,autumn,1,3,2016,Accident on I-287 Cross Westchester Expy Westb...
14,28.5,-82.6,0.0,76.0,97.0,8.0,2,Day,summer,0,2,2016,Fuel spillage on Tarpon Blvd at US-19 Commerci...
18,42.4,-83.1,0.0,63.0,93.0,10.0,1,Night,autumn,0,2,2016,Right lane closed and right hand shoulder clos...
24,40.1,-75.3,0.0,72.0,83.0,10.0,1,Day,summer,0,2,2016,Slow traffic due to accident on Front St at Fa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49976,41.8,-88.0,0.0,29.0,100.0,2.0,1,Day,winter,0,2,2016,Accident on 75th St at Janes Ave.
49979,47.9,-122.0,0.0,45.0,97.0,8.0,4,Night,winter,0,2,2016,Lane blocked and slow traffic due to accident ...
49982,41.2,-73.1,0.0,79.0,79.0,10.0,5,Day,autumn,0,2,2016,Left lane closed due to accident on CT-15 Nort...
49983,35.2,-80.8,0.0,90.0,50.0,10.0,0,Day,summer,0,2,2016,At Clement Ave - Accident.


ModuleNotFoundError: No module named 'eurybia'